In [ ]:
# %%
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

In [ ]:
# nodes/slot_collector_node.py

from typing import Literal
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from agents.restaurant.slot_collector_agent import slot_executor
from utils.message_tagging import tag_message

def slot_collector_node(state) -> Command[Literal["slot_validator"]]:
    messages = state.get("messages", [])
    
    # Get latest user input
    user_input = ""
    for msg in reversed(messages):
        if msg["role"] == "user":
            user_input = msg["content"]
            break

    # Tag the message
    tagged_msg = {
        "role": "user",
        "content": user_input,
        "meta": tag_message(user_input)
    }

    # Add it to state
    messages.append(tagged_msg)

    # Run the slot collector agent
    result = slot_executor.invoke({"input": user_input})

    user_info = {
        "name": result.get("name"),
        "phone": result.get("phone"),
        "table_number": result.get("table_number"),
        "number_of_people": result.get("number_of_people"),
    }

    return Command(
        update={
            "messages": messages,
            "user_info": user_info,
            "agent_name": "slot_collector"
        },
        goto="slot_validator"
    )
